# 6.5. 汇聚层
通常当我们处理图像时，我们希望**逐渐降低**隐藏表示的空间分辨率、聚集信息，这样随着我们在神经网络中层叠的上升，每个神经元对其敏感的感受野（输入）就越大。

而我们的机器学习任务通常会跟全局图像的问题有关（例如，“图像是否包含一只猫呢？”），所以我们最后一层的神经元应该对整个输入的全局敏感。通过逐渐聚合信息，生成越来越粗糙的映射，最终实现学习全局表示的目标，同时将卷积图层的所有优势保留在中间层。

本节将介绍汇聚（pooling）层，它具有双重目的：
1. 降低卷积层对**位置**的敏感性
2. 降低对**空间降采样表示**的敏感性。

## 6.5.1. 最大汇聚层和平均汇聚层

![avatar](../img/6_6.png)

回到本节开头提到的对象边缘检测示例，现在我们将使用卷积层的输出作为$2\times 2$最大汇聚的输入。设置卷积层输入为`X`，汇聚层输出为`Y`。无论`X[i, j]`和`X[i, j + 1]`的值相同与否，或`X[i, j + 1]`和`X[i, j + 2]`的值相同与否，汇聚层始终输出`Y[i, j] = 1`。也就是说，使用$2\times 2$最大汇聚层，即使在高度或宽度上移动一个元素，卷积层仍然可以识别到模式。

In [1]:
import torch
from torch import nn

"""
在下面的代码中的pool2d函数，我们实现汇聚层的前向传播。 这类似于 6.2节中的corr2d函数。 然而，这里我们没有卷积核，输出为输入中每个区域的最大值或平均值。
"""

def pool2d(X,pool_size,mode='max'):
    p_h,p_w=pool_size
    Y=torch.zeros((X.shape[0]-p_h+1, X.shape[1]-p_w+1 ))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            if mode == 'max':
                Y[i,j]=X[i:i+p_h,j:j+p_w].max()
            elif mode=='avg':
                Y[i,j]=X[i:i+p_h,j:j+p_w].mean()
    return Y

# 构建上图中的输入张量X，验证二维最大汇聚层的输出。
X=torch.tensor([[0.0,1.0,2.0],
                [3.0,4.0,5.0],
                [6.0,7.0,8.0]])
X.shape,pool2d(X,(2,2))

(torch.Size([3, 3]),
 tensor([[4., 5.],
         [7., 8.]]))

In [2]:
# 验证平均汇聚层。
pool2d(X,(2,2),'avg')

tensor([[2., 3.],
        [5., 6.]])

# 6.5.2. 填充和步幅
与卷积层一样，汇聚层也可以改变输出形状。和以前一样，我们可以通过填充和步幅以获得所需的输出形状。

下面，我们用深度学习框架中内置的二维最大汇聚层，来演示汇聚层中填充和步幅的使用。

In [4]:
# 首先构造了一个输入张量X，它有四个维度，其中样本数和通道数都是1。(n,c,h,w)
X=torch.arange(16,dtype=torch.float32).reshape((1,1,4,4))
X

tensor([[[[ 0.,  1.,  2.,  3.],
          [ 4.,  5.,  6.,  7.],
          [ 8.,  9., 10., 11.],
          [12., 13., 14., 15.]]]])

In [6]:
"""
默认情况下，深度学习框架中的步幅与汇聚窗口的大小相同。

因此，如果我们使用形状为(3, 3)的汇聚窗口，那么默认情况下，我们得到的步幅形状为(3, 3)。
"""
pool2d=nn.MaxPool2d(kernel_size=3)
pool2d(X)

tensor([[[[10.]]]])

In [7]:
# 填充和步幅可以手动设定。
pool2d=nn.MaxPool2d(kernel_size=3,padding=1,stride=2)
pool2d(X)

tensor([[[[ 5.,  7.],
          [13., 15.]]]])

In [8]:
# 设定一个任意大小的矩形汇聚窗口，并分别设定填充和步幅的高度和宽度。
pool2d=nn.MaxPool2d((2,3),stride=(2,3),padding=(0,1))
pool2d(X)

tensor([[[[ 5.,  7.],
          [13., 15.]]]])

## 6.5.3. 多个通道
在处理多通道输入数据时，汇聚层在每个输入通道上单独运算，而不是像卷积层一样在通道上对输入进行汇总。

这意味着汇聚层的输出通道数与输入通道数相同。

In [8]:
# 在通道维度上连结张量X和X + 1，以构建具有2个通道的输入。
# Concatenates the given sequence of :attr:`seq` tensors in the given dimension.
X=torch.cat((X,X+1),1)
X

tensor([[[[ 0.,  1.,  2.,  3.],
          [ 4.,  5.,  6.,  7.],
          [ 8.,  9., 10., 11.],
          [12., 13., 14., 15.]],

         [[ 1.,  2.,  3.,  4.],
          [ 5.,  6.,  7.,  8.],
          [ 9., 10., 11., 12.],
          [13., 14., 15., 16.]]]])

In [9]:
# 汇聚后输出通道的数量仍然是2。
pool2d=nn.MaxPool2d(kernel_size=3,padding=1,stride=2)
pool2d(X)

tensor([[[[ 5.,  7.],
          [13., 15.]],

         [[ 6.,  8.],
          [14., 16.]]]])

## 6.5.4. 小结
- 对于给定输入元素，最大汇聚层会输出该窗口内的最大值，平均汇聚层会输出该窗口内的平均值。
- 汇聚层的主要优点之一是**减轻卷积层对位置的过度敏感**。
- 我们可以指定汇聚层的填充和步幅。
- 使用最大汇聚层以及大于1的步幅，可减少空间维度（如高度和宽度）。
- 汇聚层的输出通道数与输入通道数相同。